# Chekcking if I have the requirements or not

In [1]:
# If there wasn’t already installed of these libraries in the notebook environment:
!pip install transformers torch --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 28.3 MB/s eta 0:00:00


## Improting the requriements

In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from typing import List

## The Summarizer Class for transactions summarization

In [3]:
class TransactionSummarizer:
    def __init__(self, model_name: str = "t5-small"):
        """
        Initialize the summarization pipeline.
        Args:
            model_name: HF model checkpoint for summarization.
        """
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
        self.summarizer = pipeline(
            "summarization",
            model=self.model,
            tokenizer=self.tokenizer,
            framework="pt"
        )

    def summarize(
        self,
        texts: List[str],
        max_length: int = 50,
        min_length: int = 10
    ) -> List[str]:
        """
        Generate summaries for a list of transaction remarks.
        """
        summaries = []
        for text in texts:
            result = self.summarizer(
                text,
                max_length=max_length,
                min_length=min_length,
                do_sample=False
            )
            summaries.append(result[0]['summary_text'])
        return summaries

## Quick Testing

In [4]:
# choose any model checkpoint you like
summarizer = TransactionSummarizer(model_name="t5-small")

demo_texts = [
    "Paid electricity bill for May 2025 amounting to NPR 4,500 via online portal.",
    "Grocery shopping at Kumar Supermarket, Kathmandu, total NPR 2,300"
]

print("Summaries:")
for orig, summ in zip(demo_texts, summarizer.summarize(demo_texts)):
    print(f"- Orig: {orig}\n  -> Summ: {summ}\n")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cpu
Your max_length is set to 50, but your input_length is only 22. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=11)
Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Summaries:


Your max_length is set to 50, but your input_length is only 21. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=10)
Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


- Orig: Paid electricity bill for May 2025 amounting to NPR 4,500 via online portal.
  -> Summ: electricity bill for may 2025 amounts to NPR 4,500 via online portal .

- Orig: Grocery shopping at Kumar Supermarket, Kathmandu, total NPR 2,300
  -> Summ: Grocery shopping at Kumar Supermarket, Kathmandu, total NPR 2,300 .



## Now processing through the batch

In [5]:
# If you have a pandas DataFrame of remarks:
import pandas as pd

# Example DataFrame
df = pd.DataFrame({
    'remark': [
        "Taxi fare to airport, NPR 1,200 paid via eSewa.",
        "Paid monthly Netflix subscription, NPR 850."
    ]
})

# Run summarization in batch
df['summary'] = summarizer.summarize(df['remark'].tolist())
df

Your max_length is set to 50, but your input_length is only 23. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=11)
Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Your max_length is set to 50, but your input_length is only 14. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=7)
Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


,remark,summary
0,"Taxi fare to airport, NPR 1,200 paid via eSewa.","taxi fare to airport, NPR 1,200 paid via eSewa ."
1,"Paid monthly Netflix subscription, NPR 850.","Paid monthly Netflix subscription, NPR 850."


## Batch overview helper code

In [6]:
def summarize_batch_overview(
    summarizer: TransactionSummarizer,
    texts: List[str],
    max_length: int = 60,
    min_length: int = 20
) -> str:
    """
    Concatenate a list of texts into one prompt and return a single summary.

    Args:
        summarizer: your TransactionSummarizer instance
        texts: list of remark strings
        max_length: max tokens for the joint summary
        min_length: min tokens for the joint summary

    Returns:
        A one‐paragraph summary string covering all inputs.
    """
    # Prefix each item so the model sees distinct points
    joint_input = " ".join(f"- {t}" for t in texts)
    # The summarizer expects a list:
    [overview] = summarizer.summarize(
        [joint_input],
        max_length=max_length,
        min_length=min_length
    )
    return overview

## Now testing the above function

In [7]:
# Re-use the instance you already created
summarizer = TransactionSummarizer(model_name="t5-small")

batch = [
    "Taxi fare to airport, NPR 1,200 paid via eSewa.",
    "Paid monthly Netflix subscription, NPR 850.",
    "Paid electricity bill for May 2025 amounting to NPR 4,500 via online portal."
]

overview = summarize_batch_overview(summarizer, batch)
print("Batch Overview:")
print(overview)

Device set to use cpu
Your max_length is set to 60, but your input_length is only 59. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=29)
Both `max_new_tokens` (=256) and `max_length`(=60) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Batch Overview:
taxi fare to airport, NPR 1,200 paid via eSewa . - Paid monthly Netflix subscription, 850 .


# As the summarizations was done but had been giving little unrequired summarizations

Here I have been using the LoRA Fine Tuning and RoBERTa transformer model

In [8]:
!pip install transformers peft accelerate datasets --quiet

## Now installing the requirements

In [9]:
from transformers import (
    RobertaTokenizerFast,
    EncoderDecoderModel,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer
)
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType
from datasets import load_dataset

## Load tokenizer and Model

In [10]:
# Tokenizer from RoBERTa & wrapping into encoder-decoder with GPT-2
enc_name = "roberta-base"
dec_name = "gpt2"               # small decoder
tokenizer = RobertaTokenizerFast.from_pretrained(enc_name)
tokenizer.pad_token = tokenizer.eos_token

model = EncoderDecoderModel.from_encoder_decoder_pretrained(
    enc_name, dec_name,
)
model.config.decoder_start_token_id = tokenizer.eos_token_id
model.config.eos_token_id = tokenizer.eos_token_id
model.config.pad_token_id = tokenizer.pad_token_id
model.config.vocab_size = model.config.encoder.vocab_size
model.config.max_length = 128
model.config.min_length = 30
model.config.no_repeat_ngram_size = 3
model.config.early_stopping = True
model.config.length_penalty = 2.0

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

...

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['transformer.h.0.crossattention.c_attn.bias', 'transformer.h.0.crossattention.c_attn.weight', 'transformer.h.0.crossattention.c_proj.bias', 'transformer.h.0.crossattention.c_proj.weight', 'transformer.h.0.crossattention.q_attn.bias', 'transformer.h.0.crossattention.q_attn.weight', 'transformer.h.0.ln_cross_attn.bias', 'transformer.h.0.ln_cross_attn.weight', 'transformer.h.1.crossattention.c_attn.bias', 'transformer.h.1.crossattention.c_attn.weight', 'transformer.h.1.crossattention.c_proj.bias', 'transformer.h.1.crossattention.c_proj.weight', 'transformer.h.1.crossattention.q_attn.bias', 'transformer.h.1.crossattention.q_attn.weight', 'transformer.h.1.ln_cross_attn.bias', 'transformer.h.1.ln_cross_attn.weight', 'transformer.h.10.crossattention.c_attn.bias', 'transformer.h.10.crossattention.c_attn.weight', 'transformer.h.10.crossattention.c_proj.bias', 'transformer.h.10.cros

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

## Preparing the PEFT LoRA

In [13]:
peft_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    inference_mode=False,
    r=8,               # LoRA rank
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["query", "value"]
)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 294,912 || all params: 277,747,200 || trainable%: 0.1062


## Loading the dataset and tokenizing it

In [15]:
from datasets import Dataset
import pandas as pd

df = pd.read_csv("/content/my_txns.csv")
dataset = Dataset.from_pandas(df)

def preprocess(example):
    enc = tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )
    dec = tokenizer(
        example["summary"],
        truncation=True,
        padding="max_length",
        max_length=64
    )
    enc["labels"] = dec["input_ids"]
    return enc

dataset = dataset.map(preprocess, remove_columns=["text", "summary"])
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

## Training Arguments and Trainer

In [16]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./lora-summaries",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    num_train_epochs=3,
    learning_rate=1e-4,
    fp16=True,
    logging_steps=50,
    save_total_limit=2,
    predict_with_generate=True,
    push_to_hub=False,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)
trainer.train()
trainer.save_model("./lora-summaries")

No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: rohanthapa (rohanthapa-coventry-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss


## Testing on a Batch

In [19]:
from peft import PeftModel
from transformers import pipeline

# Reload base model and apply LoRA adapters
base_model = EncoderDecoderModel.from_encoder_decoder_pretrained(enc_name, dec_name)
peft_model = PeftModel.from_pretrained(base_model, "./lora-summaries")

# Create a summarization pipeline with greedy decoding (no beam search)
summarizer = pipeline(
    "summarization",
    model=peft_model,
    tokenizer=tokenizer,
    framework="pt"
)

def summarize_text(text: str) -> str:
    """
    Generate a summary with greedy decoding (num_beams=1).
    """
    result = summarizer(
        text,
        max_length=60,
        min_length=20,
        num_beams=1,
        early_stopping=True
    )
    return result[0]["summary_text"]

# Prepare batch
batch = [
    "Taxi fare to airport, NPR 1,200 paid via eSewa.",
    "Paid monthly Netflix subscription, NPR 850.",
    "Paid electricity bill for May 2025 amounting to NPR 4,500 via online portal."
]
mega_prompt = " ".join(f"- {x}" for x in batch)

# Run greedy summarization
overview = summarize_text(mega_prompt)
print("Batch Overview Summary:", overview)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['transformer.h.0.crossattention.c_attn.bias', 'transformer.h.0.crossattention.c_attn.weight', 'transformer.h.0.crossattention.c_proj.bias', 'transformer.h.0.crossattention.c_proj.weight', 'transformer.h.0.crossattention.q_attn.bias', 'transformer.h.0.crossattention.q_attn.weight', 'transformer.h.0.ln_cross_attn.bias', 'transformer.h.0.ln_cross_attn.weight', 'transformer.h.1.crossattention.c_attn.bias', 'transformer.h.1.crossattention.c_attn.weight', 'transformer.h.1.crossattention.c_proj.bias', 'transformer.h.1.crossattention.c_proj.weight', 'transformer.h.1.crossattention.q_attn.bias', 'transformer.h.1

Batch Overview Summary: ÛÛ around change head for or for supporters For yen V choicesu horses phone 7 Ben t steering 7 critical weekend Ben inaret Congo inORY t 7 Ben immun Bucket Singh already km 7 weekend familyuallymonth for around around change horses in choices As head for or for scenario schedule As Cleveland Bears in objection taret Congo immun 7uallyLU 8StaffON already km 7 weekend familyuallymonth in 8 t 7 Benrosselage Group familyves 7 weekend familyuallymonth called immun already 7 atmosphere For 7 weekend familyuallymonth for around around change horses in choices As head for or for scenario schedule As Cleveland Bears in objection taret Congo immun 7uallyLU 8StaffON already km 7 weekend familyuallymonth in 8 t 7 Benrosselage Group familyves 7 weekend familyuallymonth called immun already 7 atmosphere For 7 weekend familyuallymonth for around around change horses in choices As head for or for scenario schedule As Cleveland Bears in objection taret Congo immun 7uallyLU 8Staf

### Now I need to make a simple project for getting things done as the model was not so correct after all.

# Written by Rohan Thapa